In [1]:
#!pip3 install jupyter_dash
#!pip3 install yfinance
#!pip install 'numpy==1.17.2'
#!pip install 'pandas==0.25.1'

import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import datetime
import glob
import settings



from dash.dependencies import Input, Output


from jupyter_dash import JupyterDash

import yfinance as yf

In [2]:
colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}

sentiment_filename_dict = {}
available_indicators = []

# Get all files with pattern 'x_sentiments.csv'
sentiment_csv_files = glob.glob('*_sentiments.csv')
print('Sentiment analysis files available: \n', sentiment_csv_files, '\n')

# Check the keyword searched for in each available x_sentiment.csv file and add it to the drop down menu option list
# Additionally, create a dictionary with keyword:filename mapping for later
for file in sentiment_csv_files:
    df = pd.read_csv(file, delimiter=',', nrows=5)
    keyword = df.keyword.unique()[0]
    print(file + ' is for keyword: ' + keyword)
    sentiment_filename_dict[keyword] = file
    available_indicators.append(keyword)
    
# Define the commodity prices that can be looked up
stock_price_options = {'#Google': 'GOOGL', \
                       '#Bitcoin': 'BTC-USD', \
                       '#tesla': 'TSLA', \
                       '#ethereum': 'ETH-USD'}

Sentiment analysis files available: 
 ['0_sentiments.csv'] 

0_sentiments.csv is for keyword: #ethereum


In [3]:
# get stocks when starting
stock_returns = {'GOOGL':pd.DataFrame(yf.download('GOOGL', period="5m", interval="1m" )),
                'BTC-USD': pd.DataFrame(yf.download('BTC-USD', period="5m", interval="1m")),
                'TSLA': pd.DataFrame(yf.download('TSLA', period="5m", interval="1m")),
                'ETH-USD': pd.DataFrame(yf.download('ETH-USD', period="5m", interval="1m"))}


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [4]:
%pwd
%ls


0_sentiments.csv             __pycache__/
Archive/                     controller.py
ClientGenerator.py           credentials.py
Dash.ipynb                   requirements_updated.txt
SendTwitterData_Script.py    sentiment_analysis_pipeline/
SentimentAnalysis_Script.py  settings.py
TwitterServer.py


In [5]:
#app = dash.Dash()
app = JupyterDash(__name__)

In [6]:
app.layout = html.Div([
    html.H1("Sentiments", style = {
        'textAlign' : 'center',
        'color': colors['text']
    }),
    # choose the twitter tags that should be fed into the sentiment analysis and query the respective stock price.
    dcc.Dropdown(
        id = "dropdown",
        options=[ {'label': i, 'value': i} for i in available_indicators]
    ),
    dcc.Graph(
        id='sentiment',
    ),
    dcc.Interval(
            id='interval-component',
            interval=5*1000, # in milliseconds
            n_intervals=0
        ),
    html.H1("Stockprices", style = {
        'textAlign' : 'center',
        'color': colors['text']
    }),
    dcc.Graph(
        id='stock prices'
    )
    
])

In [7]:
@app.callback(
    Output(component_id='stock prices', component_property='figure'),
    Input(component_id='dropdown', component_property='value'),
    Input(component_id='interval-component', component_property='n_intervals'))
def update_stocks(selected_tag, n_intervals):

    # If a tag is selected, grab the price data
    if selected_tag:
        if selected_tag in stock_price_options.keys():
            df = yf.download(stock_price_options[selected_tag], period='5m', interval='1m')            

            stock_returns[stock_price_options[selected_tag]] = stock_returns[stock_price_options[selected_tag]].append(df).drop_duplicates()
            
            fig_stocks = px.line(stock_returns[stock_price_options[selected_tag]], x=stock_returns[stock_price_options[selected_tag]].index.tz_convert(None), y="Open")
        else:
            fig_stocks = go.Figure()
        
    # If a tag is not selected, just use a generic placeholder figure
    else:
        fig_stocks = go.Figure()

    fig_stocks.update_layout(
        plot_bgcolor=colors['background'],
        paper_bgcolor=colors['background'],
        font_color=colors['text'],
        transition_duration=500)   

    return fig_stocks
    

@app.callback(
    Output(component_id='sentiment', component_property='figure'),
    Input(component_id='dropdown', component_property='value'),
    Input(component_id='interval-component', component_property='n_intervals'))
    # more inputs. variable in update_figure(input1, input2...)
    #Input('yaxis-column', 'value'),
    #Input('xaxis-type', 'value'),
    #for multiple outputs: return Out1, out2, ...
def update_figure(selected_tag, n_intervals):

    # ------- load data all 5 second (see Interval component)--------
    if selected_tag:
        filename = sentiment_filename_dict[selected_tag]
        df_sentiment = pd.read_csv(filename, delimiter=',')
        
        #parse as datetime
        df_sentiment['timestamp'] = pd.to_datetime(df_sentiment['timestamp'], unit = "s", utc = True)
    
        #change the dataframe to long format
        df_sentiment = df_sentiment.melt(id_vars = ["keyword","timestamp"],  \
                                         value_vars=["positive", "neutral", "negative"], \
                                         value_name = "count", \
                                         var_name = "sentiment")

        # ------------plot graph -----
        fig_sentiment = px.line(df_sentiment, x="timestamp", y="count", color="sentiment")

        fig_sentiment.update_layout(
            plot_bgcolor=colors['background'],
            paper_bgcolor=colors['background'],
            font_color=colors['text'],
            transition_duration=500)
        
    # If a tag is not selected, just use a generic placeholder figure
    else:
        fig_sentiment = go.Figure()
    return fig_sentiment


In [8]:
if __name__ == '__main__':
    app.run_server(port = 4060)
    #,mode="inline", 
    #pd.read_json("path...")

Dash app running on http://127.0.0.1:4060/
